In [1]:
from data import read_svm_data
from cvxopt import matrix, solvers
import numpy as np

In [2]:
training_labels, training_images = read_svm_data("training", "MNIST_ORG", [2, 3, 8, 9])
testing_labels, testing_images = read_svm_data("testing", "MNIST_ORG", [2, 3, 8, 9])

training_images.shape, training_labels.shape, testing_images.shape, testing_labels.shape

((20000, 784), (20000,), (3974, 784), (3974,))

In [3]:
# Normalize the images
training_images = training_images.astype(np.float64) / 255.0
testing_images = testing_images.astype(np.float64) / 255.0

In [4]:
# Constants
C = 100
N = training_labels.shape[0]  # number of training samples
d = training_images.shape[1]  # dimension of each sample
labels_to_classify = [2, 3, 8, 9]
classifiers = {}

In [ ]:
for label in labels_to_classify:
    N = training_labels.shape[0]
    d = training_images.shape[1]

    # Define matrix Q
    Q = np.zeros((d + N + 1, d + N + 1))
    Q[:d, :d] = np.eye(d)
    Q[d:, d:] += 1e-6 * np.eye(N + 1)

    # Define vector p
    p = np.zeros(d + N + 1)
    p[d+1:] = C

    # Define matrix A and vector c
    A = np.zeros((N, d + N + 1))
    c = -np.ones(N)

    for i in range(N):
        yn = 1 if training_labels[i] == label else -1
        A[i, :d] = -yn * training_images[i]
        A[i, d] = -yn
        A[i, d + 1 + i] = -1

    # Convert to cvxopt matrices
    Q = matrix(Q)
    p = matrix(p)
    G = matrix(A)
    h = matrix(c)

    # Solve the quadratic programming problem
    sol = solvers.qp(P=Q, q=p, G=G, h=h, A=None, b=None)

    # Extract solution
    w = np.array(sol['x'][:d]).flatten()
    b = np.array(sol['x'][d]).flatten()

    # Store the classifier
    classifiers[label] = (w, b)

In [6]:
def predict(X, classifiers):
    # Store predictions from each classifier
    predictions = {label: np.dot(X, w) + b for label, (w, b) in classifiers.items()}
    # Decide the final label based on the classifier with the maximum output
    final_predictions = np.fromiter((max(predictions, key=lambda x: predictions[x][i]) for i in range(len(X))), dtype=int)
    return final_predictions

In [8]:
predictions = predict(training_images, classifiers)
mapped_labels = np.array([label if label in labels_to_classify else None for label in training_labels])
correct_predictions = np.sum(predictions == mapped_labels)
accuracy = correct_predictions / len(training_labels)
print(f"Training Accuracy: {accuracy * 100:.2f}%")

predictions = predict(testing_images, classifiers)
mapped_labels = np.array([label if label in labels_to_classify else None for label in testing_labels])
correct_predictions = np.sum(predictions == mapped_labels)
accuracy = correct_predictions / len(testing_labels)
print(f"Testing Accuracy: {accuracy * 100:.2f}%")


Training Accuracy: 84.75%
Testing Accuracy: 85.71%
